In [1]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

import numpy as np
from sentence_transformers import SentenceTransformer


import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

scoring_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

c:\Users\kwon\anaconda3\envs\papering\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
modules.json: 100%|██████████| 341/341 [00:00<00:00, 341kB/s]
c:\Users\kwon\anaconda3\envs\papering\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kwon\.cache\huggingface\hub\models--sentence-transformers--distiluse-base-multilingual-cased-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you 

In [3]:
# MODEL = "nlpai-lab/kullm-polyglot-5.8b-v2"
# MODEL = "KT-AI/midm-bitext-S-7B-inst-v1"
# MODEL = "EleutherAI/polyglot-ko-1.3b"
MODEL = "EleutherAI/polyglot-ko-1.3b_sft10"
MODEL=r"\maywell\Synatra-42dot-1.3B_sft10"
# MODEL = "facebook/xglm-564M"
model = AutoModelForCausalLM.from_pretrained(
    "model/" + MODEL + "/checkpoint-last",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
).to(device=f"cuda")
model.eval()
tokenizer = AutoTokenizer.from_pretrained("model/" + MODEL + "/checkpoint-last")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Embedding vector 변환
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

def compute_cosine(pred, gt):
    
    # 생성된 답변 내용을 512 Embedding Vector로 변환
    pred_embed = scoring_model.encode(pred)
    gt_embed = scoring_model.encode(gt)
    
    sample_score = cosine_similarity(gt_embed, pred_embed)
    # Cosine Similarity Score가 0보다 작으면 0으로 간주
    sample_score = max(sample_score, 0)
    print('예측 : ', pred)
    print('정답 : ', gt)
    print('Cosine Similarity Score : ', sample_score)
    print('-'*20)

    return sample_score

In [5]:
train_df = pd.read_csv('./data/train.csv')

In [12]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# prompter = Prompter("kullm")

def infer(instruction="", input_text=""):
    prompt = instruction + input_text
    output = pipe(prompt, max_length=512, temperature=0.2, num_beams=5, eos_token_id=2)
    s = output[0]["generated_text"].split('A: ')[-1]
    result = s

    return result

# f"""###질문: {example['quenstion'][i]}
# ###범주: {example['category'][i]}
# ###답변: {example['answer'][i]}"""

In [13]:
import random
idx = random.randint(0, 664)
data = train_df.iloc[1, 1:3]
ans = train_df.iloc[1, 4]
q1, q2 = data
instruction = "건축, 도배에 관한 질의응답입니다. 간단하게 대답해주세요."
result1 = infer(instruction, "###Q: " + q1 + "\n###A: ")
result2 = infer(instruction, q2)
print("Q",q1)
print("A",result1)
print()
print(q2)
print(result2)

Q 내진설계의 종류 좀 알려줘
A  내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조물 전체에 지진으로부터의 영향을 분산시키기 위해 설계되었습니다. 반면, 제진구조는 지진으로 인한 충격을 줄이기 위해 주로 건물의 지지대에 설치됩니다. 또한 면진구조는 건물의 외벽에서 지진으로부터 보호받기 위한 설계 방식입니다. 이러한 내진설계의 종류는 지역의 지진 환경 및 건물의 특성에 맞게 선택되며, 안전하고 견고한 건물을 만드는 데 중요한 역할을 합니다. 따라서 각 지역의 지진 환경을 고려하여 적합한 내진설계의 종류를 선택하는 것이 중요합니다.  내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조물 전체에 지진으로부터의 영향을 분산시키기 위해 설계되었습니다. 반면, 제진구조는 지진으로 인한 충격을 줄이기 위해 주로 건물의 지지대에 설치됩니다. 또한 면진구조는 건물의 외벽에서 지진으로부터 보호받기 위한 설계 방식입니다. 이러한 내진설계의 종류는 지역의 지진 환경 및 건물의 특성에 맞게 선택되며, 안전하고 견고한 건물을 만드는 데 중요한 역할을 합니다. 따라서 각 지역의 지진 환경을 고려하여 적합한 내진설계의 종류를 선택하는 것이 중요합니다.  내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조물 전체에 지진으로부터의 영향을 분산시키기 위해 설계되었습니다. 반면, 제진구조는 지진으로 인한 충격을 줄이기 위해 주로 건물의 지지대에 설치됩니다. 또한 면진구조는 건물의 외벽에서 지진으로부터 보호받기 위한 설계 방식입니다. 이러한 내진설계의 종류는 지역의 지진 환경 및 건물의 특성에 맞게 선택되며, 안전하고 견고한 건물을 만드는 데 중요한 역할을 합니다. 따라서 각 지역의 지진 환경을 고려하여 적합한 내진설계의 종류를 선택하는 것이 중요합니다.  내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조물 전체에 지진으로부터의 영향을 분산시키기 위해 설계되었습니다

In [14]:
compute_cosine(result1, ans), compute_cosine(result2, ans)

예측 :   내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조물 전체에 지진으로부터의 영향을 분산시키기 위해 설계되었습니다. 반면, 제진구조는 지진으로 인한 충격을 줄이기 위해 주로 건물의 지지대에 설치됩니다. 또한 면진구조는 건물의 외벽에서 지진으로부터 보호받기 위한 설계 방식입니다. 이러한 내진설계의 종류는 지역의 지진 환경 및 건물의 특성에 맞게 선택되며, 안전하고 견고한 건물을 만드는 데 중요한 역할을 합니다. 따라서 각 지역의 지진 환경을 고려하여 적합한 내진설계의 종류를 선택하는 것이 중요합니다.  내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조물 전체에 지진으로부터의 영향을 분산시키기 위해 설계되었습니다. 반면, 제진구조는 지진으로 인한 충격을 줄이기 위해 주로 건물의 지지대에 설치됩니다. 또한 면진구조는 건물의 외벽에서 지진으로부터 보호받기 위한 설계 방식입니다. 이러한 내진설계의 종류는 지역의 지진 환경 및 건물의 특성에 맞게 선택되며, 안전하고 견고한 건물을 만드는 데 중요한 역할을 합니다. 따라서 각 지역의 지진 환경을 고려하여 적합한 내진설계의 종류를 선택하는 것이 중요합니다.  내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조물 전체에 지진으로부터의 영향을 분산시키기 위해 설계되었습니다. 반면, 제진구조는 지진으로 인한 충격을 줄이기 위해 주로 건물의 지지대에 설치됩니다. 또한 면진구조는 건물의 외벽에서 지진으로부터 보호받기 위한 설계 방식입니다. 이러한 내진설계의 종류는 지역의 지진 환경 및 건물의 특성에 맞게 선택되며, 안전하고 견고한 건물을 만드는 데 중요한 역할을 합니다. 따라서 각 지역의 지진 환경을 고려하여 적합한 내진설계의 종류를 선택하는 것이 중요합니다.  내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조물 전체에 지진으로부터의 영향을 분산시키기 위해 설계되었습니다. 반면, 제진구조는 지

(0.43291423, 0.40721378)

In [15]:
test_df = pd.read_csv('data/preproc_test.csv')

In [16]:
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
submission = pd.read_csv('data/sample_submission.csv')

for i, text in tqdm(enumerate(test_df['question'])):
    # pred = infer(text).split('.')[0]
    pred = infer(instruction, text)
    pred = scoring_model.encode(pred)
    submission.iloc[i, 1:] = pred


0it [00:00, ?it/s]

130it [32:50, 15.16s/it]


In [20]:
MODEL='Synatra-42dot-1.3B_sft10'

In [21]:
submission.to_csv(f"data/submission/{MODEL.split('/')[-1]}.csv", index=False)
pd.read_csv(f"data/submission/{MODEL.split('/')[-1]}.csv")

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.050423,0.021929,-0.004277,-0.012403,0.034989,0.003718,-0.001094,0.003133,-0.000515,...,-0.005100,-0.037386,0.042823,-0.028536,0.004168,-0.025020,0.075805,0.030236,0.010631,0.052760
1,TEST_001,0.020735,0.013235,0.006741,0.013161,0.050746,0.002894,-0.014682,-0.011415,-0.024977,...,0.010186,-0.000495,0.042313,-0.020366,-0.040632,0.023600,0.016050,0.003394,0.023869,0.047930
2,TEST_002,0.031516,-0.059725,-0.055009,0.004024,0.067844,-0.021349,0.010299,-0.018998,0.007556,...,-0.008242,-0.012243,0.067022,-0.021126,0.014659,0.001954,0.001254,-0.003010,-0.028776,0.082253
3,TEST_003,0.011521,-0.006158,-0.031964,0.028383,0.041894,-0.033372,-0.022815,-0.021335,-0.039976,...,-0.003490,0.005153,0.047875,-0.047296,-0.012026,0.006479,-0.013087,-0.011817,-0.011446,0.062047
4,TEST_004,0.032581,0.012349,-0.004242,-0.011176,0.062909,0.023073,0.042893,0.016032,-0.021611,...,0.028276,0.013586,0.050220,0.010649,-0.032319,-0.004558,0.026456,0.040374,-0.003893,0.067779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,TEST_125,0.041787,-0.029987,-0.005501,-0.006226,0.057556,-0.049835,0.049427,0.030020,0.022671,...,0.030844,-0.002894,0.094166,-0.034169,0.006344,-0.012093,0.045191,-0.015211,-0.026634,0.053393
126,TEST_126,0.053792,-0.048998,-0.045557,0.003684,0.074532,-0.002417,0.017463,-0.015500,-0.019457,...,-0.013017,-0.025400,0.048935,-0.016889,0.013491,0.017465,0.015076,-0.006445,0.003929,0.078474
127,TEST_127,-0.008898,0.012549,-0.032952,0.034074,0.066123,0.002786,0.005002,-0.008536,-0.068177,...,0.000075,-0.057616,0.049681,-0.015763,-0.027778,0.029018,0.048885,0.027484,-0.025435,0.049164
128,TEST_128,0.024318,-0.009040,-0.075605,0.035001,0.070618,-0.043342,0.028830,0.012641,0.001391,...,-0.038633,-0.007672,0.034521,-0.023638,0.003973,-0.015758,0.006341,0.026481,0.040047,0.036294
